In [1]:
import pandas as pd
import numpy as np
import limix
import qtl_output
import qtl_loader_utils
from qtl_snp_qc import do_snp_qc
import glob
import os
from sklearn.preprocessing import Imputer
import argparse

In [2]:
# ##loader functions
# def _ensure_dir(file_path):
#     '''Check if directory exists for output, and create it if it doesn't.'''
#     directory = os.path.dirname(file_path)
#     if not os.path.exists(directory):
#         os.makedirs(directory)

# def _get_kinship_df(kinship_filename):
#     if kinship_filename:
#         kinship_df = pd.read_csv(kinship_filename,sep='\t',index_col=0)
#     else:
#         kinship_df = None
#     return kinship_df

# def _get_samplemapping_df(sample_mapping_filename,sample_labels,key_from):
#     assert(key_from in ['iid','sample'])
#     if sample_mapping_filename:
#         mapping_df = pd.read_csv(sample_mapping_filename,sep='\t',header=None,names=['iid','sample'])
#         mapping_df.set_index(key_from,inplace=True)
#     else:
#         #assume the mapping is the identity mapping
#         identifiers = sample_labels
#         mapping_df = pd.DataFrame(data=identifiers,index=identifiers,columns=['sample'])
#     return mapping_df

# def _get_covariate_df(covariates_filename):
#     if covariates_filename:
#         covariate_df = pd.read_csv(covariates_filename,sep='\t',index_col=0)
#     else:
#         covariate_df = None
#     return covariate_df

# def _get_genotype_data(geno_prefix):
#     bim,fam,bed = limix.io.read_plink(geno_prefix,verbose=False)
#     fam.set_index('iid',inplace=True)
#     return bim,fam,bed

# def _get_annotation_df(anno_filename):
#     annotation_col_dtypes = {'feature_id':np.object,
#                          'gene_id':np.object,
#                          'gene_name':np.object,
#                          'chromosome':np.object,
#                          'start':np.int64,
#                          'end':np.int64,
#                          'strand':np.object}
#     annotation_df = pd.read_csv(anno_filename,sep='\t',index_col=0,dtype=annotation_col_dtypes)
#     return annotation_df

# def _get_phenotype_df(pheno_filename):
#     return pd.read_csv(pheno_filename,sep='\t',index_col=0)

# def do_snp_qc(snp_df, min_call_rate, min_maf, min_hwe_P):
   
#     #Determine call rate.
#     call_rate = 1-snp_df.isnull().sum()/len(snp_df.index)
#     #print(call_rate)
#     #print(call_rate < min_call_rate)
#     failed_snp_names  = list(snp_df.columns[call_rate < min_call_rate])
#     #pass_qc_snps_all = list(snp_df.columns[call_rate >= min_call_rate])
#     snp_df = snp_df.loc[:,list(snp_df.columns[call_rate >= min_call_rate])]

#     #Determine MAF.
#     genotypeCounter = np.zeros((len(snp_df.columns),3))
#     for snp in range(0, len(snp_df.columns)):
#         for sample in range(0, len(snp_df.index)):
#             if not math.isnan(snp_df.iloc[sample,snp]) :
#                 #print(snp_df.iloc[sample,snp])
#                 #print(i)
#                 #print(snp)
#                 #print(int(round(snp_df.iloc[snp,i])))
#                 genotypeCounter[snp,int(round(snp_df.iloc[sample,snp]))] +=1

#     #Here we make sure that the major allele is temporarly "coded" as 0.
#     for snp in range(0, len(snp_df.columns)):
#         if genotypeCounter[snp,0]<genotypeCounter[snp,2]:
#             tmp = genotypeCounter[snp,0]
#             genotypeCounter[snp,0] = genotypeCounter[snp,2]
#             genotypeCounter[snp,2] = tmp

#     #print(genotypeCounter)
#     maf = np.zeros((len(snp_df.columns)))
#     for snp in range(0, len(snp_df.columns)):
#         maf[snp] = ((genotypeCounter[snp,2]*2)+genotypeCounter[snp,1])/(len(snp_df.index)*2)

#     failed_snp_names.extend(list(snp_df.columns[maf < min_maf]))
#     snp_df = snp_df.loc[:,list(snp_df.columns[maf >= min_maf])]    
#     genotypeCounter = genotypeCounter[maf >= min_maf,]

#     #Determine HWE.
#     hweP = np.zeros((len(snp_df.columns)))

#     for snp in range(0, len(snp_df.columns)):
#         rare_copies = int(((genotypeCounter[snp,2])*2)+genotypeCounter[snp,1])
#         genotypes = (genotypeCounter[snp,2]+genotypeCounter[snp,1]+genotypeCounter[snp,0])
#         het_probs = np.ones((rare_copies+1))
#         # start at midpoint #

#         mid = int(round(rare_copies * (2 * genotypes - rare_copies) / (2 * genotypes)))

#         # check to ensure that midpoint and rare alleles have same parity #
#         if mid % 2 is not rare_copies % 2 :
#             mid+=1

#         curr_homr = (rare_copies - mid) / 2;
#         curr_homc = genotypes - mid - curr_homr;
#         #print(mid)
#         #print(het_probs.shape)
#         sum_values = het_probs[mid];

#         for curr_hets in range(mid, 2, -2) :
#             het_probs[curr_hets - 2] = het_probs[curr_hets] * curr_hets * (curr_hets - 1.0) / (4.0 * (curr_homr + 1.0) * (curr_homc + 1.0))
#             sum_values += het_probs[curr_hets - 2]
#             ## 2 fewer heterozygotes for next iteration -> add one rare, one common homozygote ##
#             curr_homr+=1
#             curr_homc+=1


#         curr_homr = (rare_copies - mid) / 2;
#         curr_homc = genotypes - mid - curr_homr;
#         for curr_hets in range(mid, (int(rare_copies) - 2), 2) :
#             het_probs[curr_hets + 2] = het_probs[curr_hets] * 4.0 * curr_homr * curr_homc / ((curr_hets + 2.0) * (curr_hets + 1.0))
#             sum_values += het_probs[curr_hets + 2]
#             curr_homr-=1
#             curr_homc-=1

#         p_hwe = 0.0
#         het_probs[int(genotypeCounter[snp,1])]/= sum_values
#         for index in range(0,int(rare_copies)) :
#             if index != int(genotypeCounter[snp,1]) :
#                 het_probs[index] /= sum_values
#             if het_probs[index] <= het_probs[int(genotypeCounter[snp,1])] :
#                 p_hwe += het_probs[index];

#         hweP[snp] = 1 if p_hwe > 1.0 else p_hwe;

#     failed_snp_names.extend(list(snp_df.columns[hweP < min_hwe_P]))
#     snp_df = snp_df.loc[:,list(snp_df.columns[hweP >= min_hwe_P])]

#     return snp_df.columns, failed_snp_names

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [3]:
#settings
pheno_filename = '/hps/nobackup/stegle/users/mjbonder/tools/hipsci_pipeline/limix_QTL_pipeline/geuvadis_CEU_test_data/Expression/Geuvadis_CEU_YRI_Expr.txt.gz'
anno_filename = '/hps/nobackup/stegle/users/mjbonder/tools/hipsci_pipeline/limix_QTL_pipeline/geuvadis_CEU_test_data/Expression/Geuvadis_CEU_Annot.txt'
geno_prefix = '/hps/nobackup/stegle/users/mjbonder/tools/hipsci_pipeline/limix_QTL_pipeline/geuvadis_CEU_test_data/Genotypes/Geuvadis'
window_size = 250000
min_maf = 0.05
min_hwe_P = 0.01
min_call_rate = 0.95
#cis_mode = True
cis_mode = False

output_dir = '/hps/nobackup/stegle/users/mjbonder/tools/hipsci_pipeline/limix_QTL_pipeline/geuvadis_CEU_test_data/outputTest'
chromosome='all'

covariates_filename='/hps/nobackup/stegle/users/mjbonder/tools/hipsci_pipeline/limix_QTL_pipeline/geuvadis_CEU_test_data/Expression/Geuvadis_CEU_YRI_covariates.txt'
kinship_filename='/hps/nobackup/stegle/users/mjbonder/tools/hipsci_pipeline/limix_QTL_pipeline/geuvadis_CEU_test_data/Genotypes/Geuvadis_chr1_kinship.txt'
sample_mapping_filename='/hps/nobackup/stegle/users/mjbonder/tools/hipsci_pipeline/limix_QTL_pipeline/geuvadis_CEU_test_data/Geuvadis_CEU_gte.txt'


In [4]:
# Core function to take input and run QTL tests on a given chromosome.
#Load input data files & filter for relevant data
bim,fam,bed = qtl_loader_utils.get_genotype_data(geno_prefix)
phenotype_df = qtl_loader_utils.get_phenotype_df(pheno_filename)
individual2sample_df = qtl_loader_utils.get_samplemapping_df(sample_mapping_filename,list(phenotype_df.columns),'iid')
sample2individual_df = qtl_loader_utils.get_samplemapping_df(sample_mapping_filename,list(phenotype_df.columns),'sample')
annotation_df = qtl_loader_utils.get_annotation_df(anno_filename)
phenotype_df = phenotype_df.loc[annotation_df.index.values,individual2sample_df.loc[list(set(fam.index)&set(individual2sample_df.index)),'sample'].values]

kinship_df = qtl_loader_utils.get_kinship_df(kinship_filename)    
kinship_df = kinship_df.loc[list(set(fam.index)&set(individual2sample_df.index)),list(set(fam.index)&set(individual2sample_df.index))]

covariate_df = qtl_loader_utils.get_covariate_df(covariates_filename)
covariate_df = covariate_df.loc[individual2sample_df.loc[list(set(fam.index)&set(individual2sample_df.index)),'sample'].values,]

### make sure covariates, kinship & phenotype is not changed in a later stage.

In [5]:

if(cis_mode):
    #Remove features from the annotation that are on chromosomes which are not present anyway.
    annotation_df = annotation_df[annotation_df['chromosome'].map(lambda x: x in list(set(bim['chrom'])))]
    annotation_df = annotation_df[annotation_df['chromosome'].map(lambda x: x in list(map(str, range(1, 23))))]

#Determine features to be tested
if chromosome=='all':
    feature_list = list(set(annotation_df.index)&set(phenotype_df.index))    
else:
    feature_list = list(set(annotation_df[annotation_df['chromosome']==chromosome].index)&set(phenotype_df.index))

#Array to store indices of snps tested
tested_snp_idxs = []

feature_id = feature_list[1]
print(feature_id)

ENSG00000215014


In [12]:
chrom = str(annotation_df.loc[feature_id,'chromosome'])
print(chrom)
start = annotation_df.loc[feature_id,'start']
print(start)
end = annotation_df.loc[feature_id,'end']
print(end)

#make robust to features specified back-to-front
lowest = min([start,end])
highest = max([start,end])

if (cis_mode) : 
    snpQuery = bim.query("chrom == '%s' & pos > %d & pos < %d" % (chrom, lowest-window_size, highest+window_size))
else :
    snpQuery = bim.query("(chrom == '%s' & (pos < %d | pos > %d))|chrom != '%s'" % (chrom, lowest-window_size, highest+window_size,chrom))
    snpQuery = snpQuery.iloc[snpQuery['chrom'].map(lambda x: x in list(map(str, range(1, 23))))]
    
# blocksize = 500

# print(len(snpQuery))
# if len(snpQuery) != 0:

#     print(len(snpQuery)%blocksize)
#     previous =0
#     results_df = pd.DataFrame()
#     print(results_df.shape)
#     for snpGroup in chunker(snpQuery, blocksize):
#         #Here we need to batch for cis & trans.
#         snp_idxs = snpGroup['i'].values
#         snp_names = snpGroup['snp'].values

#         tested_snp_idxs.extend(snp_idxs)

#         ##Check for NA's in feature. Remove samples with NA's

#         sample_ids = individual2sample_df.loc[:,'sample'].values

#         phenotype_ds = phenotype_df.loc[feature_id,sample_ids]
#         contains_missing_samples = any(phenotype_ds.isnull().values)
#         phenotype_ds.dropna(inplace=True)

#         #indices for relevant individuals in genotype matrix    
#         individual_ids = list(set(fam.index)&set(sample2individual_df.loc[phenotype_ds.index,'iid']))
#         individual_idxs = fam.loc[individual_ids,'i'].values

#         #subset genotype matrix, we cannot subselect at the same time, do in two steps.
#         snp_df = pd.DataFrame(data=bed[snp_idxs,:].compute().transpose(),index=fam.index,columns=snp_names)
#         snp_df = snp_df.loc[individual_ids,:]
#         print(snp_df.shape)

1
1510355
1511373
   chrom          snp   cm    pos a0 a1  i
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G  T  1
1      1  snp_1_82676  0.0  82676  G

In [12]:

pass_qc_snps_all = []
fail_qc_snps_all = []
print(snp_df.shape)
print(contains_missing_samples)

#For the updated QC where we only use one individual per genotype. We can just select sites from df to test.
if not contains_missing_samples:
    #remove snps from snp_df if they fail QC
    snp_df = snp_df.loc[:,snp_df.columns[~snp_df.columns.isin(fail_qc_snps_all)]]
    snps_to_test_df = snp_df.loc[:,snp_df.columns[~snp_df.columns.isin(pass_qc_snps_all)]]
    
    #Only do QC on relevant SNPs. join pre-QCed list and new QCed list.
    #passed_snp_names,failed_snp_names = snp_qc.do_snp_qc(snps_to_test_df)
    #File will be called snp_qc.py
    passed_snp_names,failed_snp_names = do_snp_qc(snps_to_test_df, min_call_rate, min_maf, min_hwe_P)
    snps_to_test_df = None
    
    #append snp_names and failed_snp_names
    pass_qc_snps_all.extend(passed_snp_names)
    fail_qc_snps_all.extend(failed_snp_names)
    
    snp_names  = list(snp_df.columns[snp_df.columns.isin(pass_qc_snps_all)])
    #snp_names = ['snp_1_2739933', 'snp_1_2750437', 'snp_1_2752719', 'snp_1_2753372']
    snp_df = snp_df.loc[:,snp_df.columns[snp_df.columns.isin(pass_qc_snps_all)]]

else:
    #Do snp QC for relevant section.
    passed_snp_names,failed_snp_names = do_snp_qc(snp_df, min_call_rate, min_maf, min_hwe_P)
    snp_df = snp_df.loc[:,snp_df.columns[snp_df.columns.isin(pass_qc_snps_all)]]
    
    
# if len(snp_df.columns) != 0:
#     continue

snp_matrix = snp_df.values
snp_df = None

snp_matrix = Imputer(missing_values=np.nan,strategy='mean',axis=0,copy=False).fit_transform(snp_matrix)

if kinship_df is not None:
    kinship_mat = kinship_df.loc[individual_ids,individual_ids].values
else:
    kinship_mat = None

#map individual_ids to samples
sample_ids = individual2sample_df.loc[individual_ids,'sample'].values
phenotype = phenotype_ds.loc[sample_ids].values

#generate covariate matrix
if covariate_df is not None:
    cov_matrix = np.concatenate([np.ones((len(sample_ids),1)),covariate_df.loc[sample_ids,:].values],axis=1)
else:
    cov_matrix = None

#fit modelrun
LMM = limix.qtl.qtl_test_lmm(snp_matrix, phenotype,K=kinship_mat,covs=cov_matrix)

#add these results to qtl_results

temp_df = pd.DataFrame(index = range(len(snp_names)),columns=['feature_id','snp_id','p_value','beta','n_samples'])
temp_df['snp_id'] = snp_names
temp_df['feature_id'] = feature_id
temp_df['beta'] = LMM.getBetaSNP()[0]
temp_df['p_value'] = LMM.getPv()[0]
temp_df['n_samples'] = sum(~np.isnan(phenotype))
results_df = results_df.append(temp_df)
print(temp_df)
# output_writer.add_result_df(temp_df)

(90, 151)
False
          feature_id         snp_id   p_value      beta  n_samples
0    ENSG00000215014  snp_2_8980472  0.247541  0.118171         90
1    ENSG00000215014  snp_2_8988660  0.561975  0.085046         90
2    ENSG00000215014  snp_2_8996746  0.561975  0.085046         90
3    ENSG00000215014  snp_2_9007398  0.775673  0.045281         90
4    ENSG00000215014  snp_2_9041284  0.894102  0.021493         90
5    ENSG00000215014  snp_2_9061326  0.557266  0.104928         90
6    ENSG00000215014  snp_2_9064878  0.894102  0.021493         90
7    ENSG00000215014  snp_2_9071427  0.697416  0.059178         90
8    ENSG00000215014  snp_2_9082673  0.557266  0.104928         90
9    ENSG00000215014  snp_2_9094995  0.894102  0.021493         90
10   ENSG00000215014  snp_2_9167124  0.965390  0.007989         90
11   ENSG00000215014  snp_2_9228515  0.369972  0.102101         90
12   ENSG00000215014  snp_2_9233610  0.764660 -0.037929         90
13   ENSG00000215014  snp_2_9239563  0.647314 

In [13]:
results_df = results_df.append(temp_df)
print(results_df)
results_df.empty

         feature_id         snp_id   p_value      beta  n_samples
0   ENSG00000215014  snp_1_1276077  0.933442  0.009569         90
1   ENSG00000215014  snp_1_1287922  0.027078 -0.523502         90
2   ENSG00000215014  snp_1_1299633  0.326317  0.206344         90
3   ENSG00000215014  snp_1_1308387  0.326317  0.206344         90
4   ENSG00000215014  snp_1_1469782  0.135740 -0.371984         90
5   ENSG00000215014  snp_1_1526799  0.198926  0.215296         90
6   ENSG00000215014  snp_1_1527952  0.087492  0.210162         90
7   ENSG00000215014  snp_1_1535700  0.357051  0.118735         90
8   ENSG00000215014  snp_1_1545289  0.215915  0.160522         90
9   ENSG00000215014  snp_1_1562536  0.420513 -0.140472         90
10  ENSG00000215014  snp_1_1584102  0.826237  0.025858         90
11  ENSG00000215014  snp_1_1590681  0.936862  0.009429         90
12  ENSG00000215014  snp_1_1602536  0.835972 -0.025113         90
13  ENSG00000215014  snp_1_1615348  0.871757 -0.019827         90
14  ENSG00

False